In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.TrashClassifier import TrashClassifier
from ai.DataLoader import DataLoader
from env import *

EPOCHS = 100
ETA = 1e-2

In [2]:
model = TrashClassifier()
model = model.cuda()

FeatureCNN was loaded.


In [3]:
train_dloader = DataLoader(TRASH_TRAIN_DATA_PATH, TRASH_CAT)
valid_dloader = DataLoader(TRASH_VALID_DATA_PATH, TRASH_CAT)

Number of data batch: 793
Number of batch: 100
Number of data batch: 65
Number of batch: 9


In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [5]:
top_valid_acc = 0.0

for e in range(EPOCHS):
    
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    for x_batch, y_batch in train_dloader.next_batch():
        x_batch = torch.FloatTensor(x_batch).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        logps = model(x_batch)
        loss = criterion(logps, y_batch)
        
        with torch.no_grad():
            train_loss += loss.item()
            ps = torch.exp(logps)
            
            ps_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_acc += torch.mean(equal.type(torch.FloatTensor))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 10 == 0:
        with torch.no_grad():
            model.eval()

            for x_batch, y_batch in valid_dloader.next_batch():
                x_batch = torch.FloatTensor(x_batch).cuda()
                y_batch = torch.LongTensor(y_batch).cuda()

                logps = model(x_batch)
                loss = criterion(logps, y_batch)

                valid_loss += loss.item()

                ps = torch.exp(logps)
                ps_k, cls_k = ps.topk(1, dim=1)
                equal = cls_k == y_batch.view(*cls_k.size())
                valid_acc += torch.mean(equal.type(torch.FloatTensor))

            train_loss /= len(train_dloader)
            train_acc /= len(train_dloader)
            valid_loss /= len(valid_dloader)
            valid_acc /= len(valid_dloader)

            print(f"Epochs: {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}")
            print(f"Train acc: {train_acc:.8f}")
            print(f"Valid loss: {valid_loss:.8f}")
            print(f"Valid acc: {valid_acc:.8f}")
            
            if top_valid_acc < valid_acc:
                top_valid_acc = valid_acc
                model.save(CLF_CKPT_PATH)

            model.train()

Epochs: 10/100
Train loss: 0.05320231
Train acc: 0.98843747
Valid loss: 0.56498208
Valid acc: 0.90277779
Classifier was saved.
Epochs: 20/100
Train loss: 0.03874812
Train acc: 0.99218750
Valid loss: 1.29634329
Valid acc: 0.84722221
Epochs: 30/100
Train loss: 0.01185430
Train acc: 0.99656248
Valid loss: 1.02933919
Valid acc: 0.90625000
Classifier was saved.
Epochs: 40/100
Train loss: 0.13182294
Train acc: 0.98468751
Valid loss: 1.27608718
Valid acc: 0.88194442
Epochs: 50/100
Train loss: 0.02596139
Train acc: 0.99687499
Valid loss: 1.81033944
Valid acc: 0.86111110
Epochs: 60/100
Train loss: 0.01388910
Train acc: 0.99750000
Valid loss: 3.97678422
Valid acc: 0.87152779
Epochs: 70/100
Train loss: 0.00716067
Train acc: 0.99781251
Valid loss: 2.25382481
Valid acc: 0.86458331


KeyboardInterrupt: 